In [ ]:
import stanza
import glob
import os
import re
from lxml import etree
import csv
import shutil



In [ ]:
#stanza.download('fr')

nlp = stanza.Pipeline(lang='fr', processors='tokenize,mwt,pos,lemma,depparse')
#doc = nlp('Barack Obama est né à Hawaii.')

In [ ]:
dir_resolco_ac_norm="../resolco_vf/**/**/copiesNormees/ac/"


print("creation du dic pour le site transcription"+dir_resolco_ac_norm)

copies_transcript=[]

files=glob.glob(dir_resolco_ac_norm+"/*.ac")
cpt=0
for file_ac in files:
    
    path_classe=re.sub(r'^(.*)\/copiesNormees\/ac\/((EC|CO|UN)-(.+?)-[^\/]+\-R[0-9]+\-V[123]\_N).*$',r"\1",file_ac)
    idTexte=re.sub(r"^.*\/((EC|CO|UN)[^\/]+\-R[0-9]+\-V[123]\_N).*$",r"\1",file_ac)
    
    print("parsing POS tag stanza du fichier : "+file_ac)
    
    ac=open(file_ac,'r',encoding='utf-8').read()
    ac=re.sub('\n','',ac)
    doc=nlp(ac)
    
    #print(doc.sentences)
    lines=[f'{word.id}\t{word.text}\t{word.lemma if word.lemma else "_"}\t{word.upos}\t{word.pos}\t{word.feats if word.feats else "_"}\t{word.deprel}\t{word.head}\t{word.start_char}\t{word.end_char}' for sent in doc.sentences for word in sent.words]
        
    
    '''OUT'''
    rep_stanza=path_classe+"/copiesNormees/stanza/"
    #if not os.path.exists(rep_stanza):
    #   os.mkdir(rep_stanza)
    file_out=rep_stanza+idTexte+"_stanzapos.txt"
    
    print("creation du fichier out : "+file_out)
    
    with open(file_out, 'w') as f:
        for line in lines:
            f.write("%s\n" % line)
    
    cpt+=1
print(cpt)
    
    #print(ac)

In [ ]:
dir_aa="../resolco_vf/**/**/copiesAnnotees/annotations_gold/"
files=glob.glob(dir_aa+"/*.aa")
cpt=0

def getkeyStart(elem):
    return float(elem.find('positioning/start/singlePosition').get('index'))

for file_aa in files:
    
    path_classe=re.sub(r'^(.*)\/copiesAnnotees\/annotations_gold\/((EC|CO|UN)-(.+?)-[^\/]+\-R[0-9]+\-V[123]\_N).*$',r"\1",file_aa)
    idTexte=re.sub(r"^.*\/((EC|CO|UN)[^\/]+\-R[0-9]+\-V[123]\_N).*$",r"\1",file_aa)
    
    file_stanza=path_classe+"/copiesNormees/stanza/"+idTexte+"_stanzapos.txt"
    
    newFile = path_classe+"/copiesNormees/csv_stanza/"+idTexte+'_withAnnotations.csv'
    if not os.path.exists(path_classe+"/copiesNormees/csv_stanza/"):
        os.mkdir(path_classe+"/copiesNormees/csv_stanza/")
    outFile = open(newFile, mode="w", encoding="utf-8")
    out=csv.writer(outFile, delimiter='\t')
    
    annots=[]
    print("parsing du fichier "+file_aa)
    
    aa = etree.parse(file_aa)
    root = aa.getroot()
    units = root.findall('unit')
    #tri des "Units" par leur offset de fin
    units[:] = sorted(units, key=getkeyStart)
    for element in units:
        typUnit = element.find('characterisation/type')

        if typUnit.text == "phraseConsigne":
            if feature is not None:		
                startPosition = element.find('positioning/start/singlePosition')
                endPosition = element.find('positioning/end/singlePosition')
                ofsStart = int(startPosition.get('index'))
                ofsEnd = int(endPosition.get('index'))
                annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":"consigne",
                    "typ":"consigne"
                })
        elif typUnit.text == "mod":
            feature = element.find('characterisation/featureSet/feature[@name="nature"]')
            if feature is not None:		
                startPosition = element.find('positioning/start/singlePosition')
                endPosition = element.find('positioning/end/singlePosition')
                ofsStart = int(startPosition.get('index'))
                ofsEnd = int(endPosition.get('index'))
                annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":str(feature.text),
                    "typ":"modif"
                })
        elif typUnit.text == "Err_Orthographe":
            feature = element.find('characterisation/featureSet/feature[@name="version originale"]')
            if feature is not None:				
                startPosition = element.find('positioning/start/singlePosition')
                endPosition = element.find('positioning/end/singlePosition')
                ofsStart = int(startPosition.get('index'))
                ofsEnd = int(endPosition.get('index'))
                alt = ""
                if element.find('characterisation/featureSet/feature[@name="correction_2"]').text is not None: 
                    alt = " -- alternatives : "+element.find('characterisation/featureSet/feature[@name="correction_2"]').text
                if element.find('characterisation/featureSet/feature[@name="correction_3"]').text is not None: 
                    alt = alt+" OR "+element.find('characterisation/featureSet/feature[@name="correction_3"]').text
                if element.find('characterisation/featureSet/feature[@name="correction_4"]').text is not None: 
                    alt = alt+" OR "+element.find('characterisation/featureSet/feature[@name="correction_4"]').text
                annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":str(feature.text)+alt,
                    "typ":"ortho"
                })
        elif typUnit.text == "maillon_Elle":
            feature = element.find('characterisation/featureSet/feature[@name="groupe"]')
            incertitudeD = element.find('characterisation/featureSet/feature[@name="incertitude sur la délimitation"]')
            incertitudeR = element.find('characterisation/featureSet/feature[@name="incertitude sur le rattachement"]')
            if feature is not None:		
                startPosition = element.find('positioning/start/singlePosition')
                endPosition = element.find('positioning/end/singlePosition')
                ofsStart = int(startPosition.get('index'))
                ofsEnd = int(endPosition.get('index'))
                annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":"G"+feature.text+"_incertD"+incertitudeD.text+"_incertR"+incertitudeR.text,
                    "typ":"maillonElle"
                })
        elif typUnit.text == "maillon_Il":
            feature = element.find('characterisation/featureSet/feature[@name="groupe"]')
            incertitudeD = element.find('characterisation/featureSet/feature[@name="incertitude sur la délimitation"]')
            incertitudeR = element.find('characterisation/featureSet/feature[@name="incertitude sur le rattachement"]')
            if feature is not None:		
                startPosition = element.find('positioning/start/singlePosition')
                endPosition = element.find('positioning/end/singlePosition')
                ofsStart = int(startPosition.get('index'))
                ofsEnd = int(endPosition.get('index'))
                annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":"G"+feature.text+"_incertD"+incertitudeD.text+"_incertR"+incertitudeR.text,
                    "typ":"maillonIl"
                })
        elif typUnit.text == "maillon_lesEnfants":
            feature = element.find('characterisation/featureSet/feature[@name="groupe"]')
            incertitudeD = element.find('characterisation/featureSet/feature[@name="incertitude sur la délimitation"]')
            incertitudeR = element.find('characterisation/featureSet/feature[@name="incertitude sur le rattachement"]')
            if feature is not None:		
                startPosition = element.find('positioning/start/singlePosition')
                endPosition = element.find('positioning/end/singlePosition')
                ofsStart = int(startPosition.get('index'))
                ofsEnd = int(endPosition.get('index'))
                membre=str(element.find('characterisation/featureSet/feature[@name="commentaire"]').text)
                if "membre" in membre:
                    annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":"G"+feature.text+"_incertD"+incertitudeD.text+"_incertR"+incertitudeR.text,
                    "typ":"maillonMembreLesEnfants"
                })
                else:
                    annots.append({
                    "start":ofsStart,
                    "end":ofsEnd,
                    "txt":"G"+feature.text+"_incertD"+incertitudeD.text+"_incertR"+incertitudeR.text,
                    "typ":"maillonLesEnfants"
                })
                    
    for l in open(file_stanza,'r'):
        l=l.rstrip("\n")
        tal = l.split("\t")
        if re.search(r"^[0-9]+",tal[0]): # si la première valeur est un nombre alors la ligne correspond à l'analyse d'un token

            if re.search(r"^[0-9]+",tal[8]) and re.search(r"^[0-9]+",tal[9]): #si les ofsets sont renseignés
                ofsStart = int(tal[8]) 
                ofsEnd = int(tal[9])
                consigne = "\t_" # variable qui indique si le token est dans une des phrases consigne
                mod = "\t_\t_" # variable mod contient 2 valeurs indiquant le type de modification dans laquelle est inclu le token et l'ofset (pour comparer)
                err = "\t_\t_\t_" # variable err contient 2 valeurs séparées par une tabulation et qui sera coller à la fin de la ligne pour chaque token. La valeur 1 indique si le token est inclu dans une erreur (1)  et la valeur 2 fournit le texte original (avant normalisation orthographique)
                elle = "\t_\t_" # variable elle contient 1 valeur indiquant si le token est inclu dans un maillon Elle. La valeur indique si c'est un groupe, s'il y a eu incertitude avec la syntaxe : (is|in)GOui_incertDOui_incertRNon. Is correspond aux token qui sont un maillon. In pour les token qui sont contenus dans un maillon.
                il = "\t_\t_" # idem pour les maillons Il
                lesenf = "\t_\t_" # idem pour les maillons Les Enfants
                membre="\t_\t_"
                for i in range(0, len(annots)):
                    ofset = str(annots[i]["start"])+"-"+str(annots[i]["end"])
                    #print(annots[i]["typ"]+annots[i]["txt"]+ofset)
                    #3 situations, la sortie distingue les cas où l'annotation est le token (is_maillonElle) des cas où le token est inclus dans une annotation (in_maillonElle)  (peut être utiliser le système BIO : Begin, inside, outside)
                    if ofsStart == annots[i]["start"] and ofsEnd == annots[i]["end"]:#si le token est une annotation
                        if annots[i]["typ"] == "ortho":
                            err = "\tErr\t"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "modif":
                            mod = "\tis"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "maillonElle":
                            elle = "\tis_"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "maillonIl":
                            il = "\tis_"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "maillonLesEnfants":
                            lesenf = "\tis_"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "maillonMembreLesEnfants":
                            membre = "\tis_"+annots[i]["txt"]+"\t"+ofset

                    elif ofsStart >= annots[i]["start"] and ofsEnd <= annots[i]["end"]:#si le token est inclus dans une annotation
                        if annots[i]["typ"] == "consigne":
                            consigne = "\tconsigne"
                        elif annots[i]["typ"] == "ortho":
                            err = "\tErr\t"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "modif":
                            mod = "\tin_"+annots[i]["txt"]+"\t"+ofset											
                        elif annots[i]["typ"] == "maillonElle":
                            elle = "\tin_"+annots[i]["txt"]+"\t"+ofset								
                        elif annots[i]["typ"] == "maillonIl":
                            il = "\tin_"+annots[i]["txt"]+"\t"+ofset								
                        elif annots[i]["typ"] == "maillonLesEnfants":
                            lesenf = "\tin_"+annots[i]["txt"]+"\t"+ofset
                        elif annots[i]["typ"] == "maillonMembreLesEnfants":
                            membre = "\tin_"+annots[i]["txt"]+"\t"+ofset								

                    elif ofsStart <= annots[i]["start"] and ofsEnd >= annots[i]["end"]:#si le token inclut une annotation
                            if annots[i]["typ"] == "ortho":
                                err = "\tErr\t"+annots[i]["txt"]+"\t"+ofset
                            elif annots[i]["typ"] == "modif":
                                mod = "\t"+annots[i]["txt"]+"\t"+ofset											
                            elif annots[i]["typ"] == "maillonElle":
                                elle = "\t"+annots[i]["txt"]+"\t"+ofset								
                            elif annots[i]["typ"] == "maillonIl":
                                il = "\t"+annots[i]["txt"]+"\t"+ofset								
                            elif annots[i]["typ"] == "maillonLesEnfants":
                                lesenf = "\t"+annots[i]["txt"]+"\t"+ofset
                            elif annots[i]["typ"] == "maillonMembreLesEnfants":
                                membre = "\t"+annots[i]["txt"]+"\t"+ofset

                output=l+consigne+mod+err+elle+il+lesenf+membre
                output=output.split("\t")
                out.writerow(output)
            else:
                output=l+"\t_\t_\t_\t_\t_\t \t \t \t \t "
                output=output.split("\t")
                out.writerow(output)
        elif l == "" :
            out.writerow("\n")
    annots.clear
    print(newFile)